# Setup Script
To be run after new videos are added

Based on the article [Guide to Build Video Classification Model](https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/)


In [1]:
!pip3 install opencv-python

### Importing Libraries

In [1]:
import os, os.path
import cv2  # for caputring videos
import math # for mathematical operations
import matplotlib.pyplot as plt # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image # for preprocessing the images
import numpy as np # for mathematical operations
from keras.utils import np_utils
from numpy import genfromtxt
from skimage.transform import resize # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

### Storing the name of videos in a dataframe
First bit opens the file, reads the file, and splits them by "enter"

Second bit creates a dataframe and stores the above extracted video names under "train['video_name']"

In [2]:
# open the .txt file which have names of training videos
# f = open("trainlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split("\n")
# print(videos)

# creating a dataframe having video names
# train = pd.DataFrame()
# train['video_name'] = videos
# train

 # Turn the csv files into dataframes
 Things to prep:
 1. Create folder with the name "training_csv"
 2. Create files with proper naming

In [31]:
path="dataPoints/"

convertFile = pd.DataFrame()

for filename in glob(os.path.join(path, '*.csv')):    
    file = filename.split("/")[1]
    convertFile = pd.read_csv(path + file, header=None)
    
#     locals()['dict{}'.format("_" + file.split(".csv")[0])] = {}
    D = {}
        
    size = len(convertFile[0]) // 12
    
    for i in range((len(convertFile) + 12 - 1) // 12):
        data = convertFile.to_numpy()[i:i+12]
        data = np.delete(data, 0, 1)
        data = np.reshape(data, (24, 1))
#         locals()['dict{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data
        D = data
#     print(D)
    
#     for i in locals()['dict{}'.format("_" + file.split(".csv")[0])]:
#         combine = locals()['dict{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
        
#         if int(i) < len(locals()['dict{}'.format("_" + file.split(".csv")[0])]):
#             j = int(i)+1
#             combineTwo = locals()['dict{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]

    for i in D:
        combine = D
        if int(i) < len(D):
            j = int(i)+1
            combineTwo = D.format(j)

# print(dict_training_file_1_b)
# print(dict_training_file_1_b["0"])
# print(dict_training_file_1_b["0"].shape)



In [18]:
#     for i in range(0, size-1):
#         data = np.append(data, [convertFile.loc[i, :].to_numpy()], axis=0)    
        
    
    
#     size = math.floor(len(convertFile))//12
    
#     for i in range(1, size):
#         trainFrame[i] = convertFile[i]
    
    
#     print(trainFrame[0])
    
#     splitArray = np.array_split(convertFile, size)
    
#     print(splitArray[0][0])
    
    
#     print(convertFile)
#     np.array_split(convertFile, size)
    
#     for i in range(0, size):
#         trainFrame[i] = convertFile[i]
    
#     print(trainFrame[0])
    
#     print(np.array_split(convertFile, size))
#         print(np.array_split(trainFrame_i, size))
    
    
#     for j in range (len(trainFrame_i)):
#         trainFrame_i[j].reshape((14,1))
    
#     for j in range (len(trainFrame_i)):
#         trainFrame_j[j] = trainFrame_i[j]
#         trainFrame_j[j].append(trainFrame_i[j+1])
#         trainFrame_j[j].append(trainFrame_i[j+2])

### Create a similar dataframe for the test videos
Does the same thing as above but for the test video names

In [8]:
# # open the .txt file which have names of test videos
# f = open("testlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split('\n')

# # creating a dataframe having video names
# test = pd.DataFrame()
# test['video_name'] = videos
# test

# Do the same for test videos

In [ ]:
files = os.listdir(dir) #dir of where we will place folder, to be replaced with actual name

for i in range (len(files)):
    testFrame_i = pd.read_csv('training_file_', i, '.csv')
    testFrame_i = testFrame.to_numpy()
    
    size = math.floor((len(testFrame_i)/12 * (10 ** 0))/ (10 ** 0))
    testFrame_i.split(testFrame_i, size)
    
    for j in range (len(testFrame_i)):
        testFrame_i[j].reshape((14,1))
    
    for j in range (len(trainFrame_i)):
        testFrame_j[j] = testFrame_i[j]
        testFrame_j[j].append(testFrame_i[j+1])
        testFrame_j[j].append(testFrame_i[j+2])

### Adding the tags
For both training and test sets: grabs the tag (0 or 1/good or bad serve) from the text file and stores in the respective dataframe.

In [5]:
# # creating tags for training videos
# train_video_tag = []
# for i in range(train.shape[0]):
#     train_video_tag.append(train['video_name'][i].split('_')[1].split('.')[0])
    
# train['tag'] = train_video_tag

# # creating tags for test videos
# test_video_tag = []
# for i in range(test.shape[0]):
#     test_video_tag.append(test['video_name'][i].split('_')[1].split('.')[0])
    
# test['tag'] = test_video_tag

In [9]:
#This is what the test dataframe looks like 
# test

In [10]:
# This is what the train dataframe looks like 
# train

# Getting the tags
This is assuming this is first column of csv file

In [ ]:
train_video_tag = []
for i in range(len(files)):
    train_video_tag.append(trainFrame_i[0][0])

### Frame Extraction and Storing
All frames are put in a folder named train_1

In [11]:
# # open the .txt file which have names of training videos
# f = open("trainlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split("\n")

# train = pd.DataFrame()
# train['video_name'] = videos

# # storing the frames from training videos
# for i in tqdm(range(len(file))):
#     count = 0
#     videoFile = train['video_name'][i]
#     cap = cv2.VideoCapture('myVids/'+videoFile)   # capturing the video from the given path
#     frameRate = cap.get(5) #frame rate
#     x=1
#     while(cap.isOpened()):
#         frameId = cap.get(1) #current frame number
#         ret, frame = cap.read()
#         if (ret != True):
#             break
#         if (frameId % math.floor(frameRate) == 0):
#             # storing the frames in a new folder named train_1
#             print("this is the split")
#             print(videoFile.split(" ")[0]) #since my videos aren't in separate folder, no need to split based on ("/")
            
#             #need to make sure that the train_1 has been created
#             filename ='train_1/' + videoFile.split(' ')[0] +"_frame%d.jpg" % count;count+=1
#             cv2.imwrite(filename, frame)
#     cap.release()

In [ ]:
# 

### .csv file
Get the names of all the frames with their corresponding tag and put the info into a .csv file

In [9]:
# #getting the names of all the images
# images = glob("train_1/*.jpg")
# train_image = []
# train_class = []

# for i in tqdm(range(len(images))):
#     # creating the image name
#     train_image.append(images[i].split("\\")[1])  #uncomment this for windows
#     #train_image.append(images[i].split("/")[1])  #uncomment this for mac
#     # creating the class of image
#     train_class.append(images[i].split("\\")[1].split(".")[0].split("_")[1]) #uncomment this for windows
#     #train_class.append(images[i].split("/")[1].split(".")[0].split("_")[1]) #uncomment this for mac
    
# # storing the images and their class in a dataframe
# train_data = pd.DataFrame()
# train_data['image'] = train_image
# train_data['class'] = train_class

# # converting the dataframe into csv file 
# train_data.to_csv('myVids/train_new.csv',header=True, index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<?, ?it/s]
